In [1]:
cd("/home/alir/Oceananigans.jl/")

In [2]:
using Pkg
pkg"activate ."

Activating environment at `~/Oceananigans.jl/Project.toml`


In [22]:
using Statistics
using OffsetArrays, FFTW
using Oceananigans, Oceananigans.Solvers

using Oceananigans: NoPenetrationBC, fill_halo_regions!
using Oceananigans.TimeSteppers: _compute_w_from_continuity!

In [5]:
@inline δx_caa(i, j, k, f) = @inbounds f[i+1, j, k] - f[i, j, k]
@inline δy_aca(i, j, k, f) = @inbounds f[i, j+1, k] - f[i, j, k]
@inline δz_aac(i, j, k, f) = @inbounds f[i, j, k+1] - f[i, j, k]

@inline ∂x_caa(i, j, k, Δx,  f) = δx_caa(i, j, k, f) / Δx
@inline ∂y_aca(i, j, k, Δy,  f) = δy_aca(i, j, k, f) / Δy
@inline ∂z_aac(i, j, k, ΔzF, f) = δz_aac(i, j, k, f) / ΔzF[k]

@inline ∂x²(i, j, k, Δx, f)       = (∂x_caa(i, j, k, Δx, f)  - ∂x_caa(i-1, j, k, Δx, f))  / Δx
@inline ∂y²(i, j, k, Δy, f)       = (∂y_aca(i, j, k, Δy, f)  - ∂y_aca(i, j-1, k, Δy, f))  / Δy
@inline ∂z²(i, j, k, ΔzF, ΔzC, f) = (∂z_aac(i, j, k, ΔzF, f) - ∂z_aac(i, j, k-1, ΔzF, f)) / ΔzC[k]

@inline div_ccc(i, j, k, Δx, Δy, ΔzF, u, v, w) = ∂x_caa(i, j, k, Δx, u) + ∂y_aca(i, j, k, Δy, v) + ∂z_aac(i, j, k, ΔzF, w)

@inline ∇²(i, j, k, Δx, Δy, ΔzF, ΔzC, f) = ∂x²(i, j, k, Δx, f) + ∂y²(i, j, k, Δy, f) + ∂z²(i, j, k, ΔzF, ΔzC, f)

∇² (generic function with 1 method)

In [6]:
function grid(zF)
    Nz = length(zF) - 1
    ΔzF = [zF[k+1] - zF[k] for k in 1:Nz]
    zC = [(zF[k] + zF[k+1]) / 2 for k in 1:Nz]
    ΔzC = [zC[k+1] - zC[k] for k in 1:Nz-1]
    return zF, zC, ΔzF, ΔzC
end

grid (generic function with 1 method)

In [7]:
arch = CPU()
Nx = Ny = 8
zF = [1, 2, 4, 7, 11, 16, 22, 29, 37]

9-element Array{Int64,1}:
  1
  2
  4
  7
 11
 16
 22
 29
 37

In [8]:
Lx, Ly, Lz = 100, 100, zF[end]
Δx, Δy = Lx/Nx, Ly/Ny

(12.5, 12.5)

In [9]:
Nz = length(zF) - 1
zF, zC, ΔzF, ΔzC = grid(zF)

# Need some halo regions.
ΔzF = OffsetArray([ΔzF[1], ΔzF...], 0:Nz)
ΔzC = [ΔzC..., ΔzC[end]]

# Useful for broadcasting z operations
ΔzC = reshape(ΔzC, (1, 1, Nz));

In [10]:
# Temporary hack: Useful for reusing fill_halo_regions! and BatchedTridiagonalSolver
# which only need Nx, Ny, Nz.
fake_grid = RegularCartesianGrid(size=(Nx, Ny, Nz), length=(Lx, Ly, Lz))

RegularCartesianGrid{Float64}
domain: x ∈ [0.0, 100.0], y ∈ [0.0, 100.0], z ∈ [-37.0, 0.0]
  resolution (Nx, Ny, Nz) = (8, 8, 8)
   halo size (Hx, Hy, Hz) = (1, 1, 1)
grid spacing (Δx, Δy, Δz) = (12.5, 12.5, 4.625)

In [11]:
#####
##### Generate batched tridiagonal system coefficients and solver
#####

function λi(Nx, Δx)
    is = reshape(1:Nx, Nx, 1, 1)
    @. (2sin((is-1)*π/Nx) / Δx)^2
end

function λj(Ny, Δy)
    js = reshape(1:Ny, 1, Ny, 1)
    @. (2sin((js-1)*π/Ny) / Δy)^2
end

kx² = λi(Nx, Δx);
ky² = λj(Ny, Δy);

In [12]:
# Lower and upper diagonals are the same
ld = [1/ΔzF[k] for k in 1:Nz-1]
ud = copy(ld)

# Diagonal (different for each i,j)
@inline δ(k, ΔzF, ΔzC, kx², ky²) = - (1/ΔzF[k-1] + 1/ΔzF[k]) - ΔzC[k] * (kx² + ky²)

d = zeros(Nx, Ny, Nz)
for i in 1:Nx, j in 1:Ny
    d[i, j, :] .= [-1/ΔzF[1], [δ(k, ΔzF, ΔzC, kx²[i], ky²[j]) for k in 2:Nz]...]
end

In [42]:
# Random right hand side
Ru = CellField(Float64, arch, fake_grid)
Rv = CellField(Float64, arch, fake_grid)
Rw = CellField(Float64, arch, fake_grid)

interior(Ru) .= rand(Nx, Ny, Nz)
interior(Rv) .= rand(Nx, Ny, Nz)
interior(Rw) .= zeros(Nx, Ny, Nz)
U = (u=Ru, v=Rv, w=Rw)

uv_bcs = HorizontallyPeriodicBCs()
w_bcs = HorizontallyPeriodicBCs(top=NoPenetrationBC(), bottom=NoPenetrationBC())

fill_halo_regions!(Ru.data, uv_bcs, arch, fake_grid)
fill_halo_regions!(Rv.data, uv_bcs, arch, fake_grid)

_compute_w_from_continuity!(U, fake_grid)

fill_halo_regions!(Rw.data,  w_bcs, arch, fake_grid)

R = zeros(Nx, Ny, Nz)
for i in 1:Nx, j in 1:Ny, k in 1:Nz
    R[i, j, k] = div_ccc(i, j, k, Δx, Δy, ΔzF, Ru.data, Rv.data, Rw.data)
end

@show sum(R)  # div(R) should be zero by construction.

F = zeros(Nx, Ny, Nz)
F = ΔzC .* R;  # RHS needs to be multiplied by ΔzC

sum(R) = -1.1102230246251565e-16


In [43]:
#####
##### Solve system
#####

F̃ = fft(F, [1, 2])

btsolver = BatchedTridiagonalSolver(arch, dl=ld, d=d, du=ud, f=F̃, grid=fake_grid)

ϕ̃ = zeros(Complex{Float64}, Nx, Ny, Nz)
solve_batched_tridiagonal_system!(ϕ̃, arch, btsolver)

ϕ = CellField(Float64, arch, fake_grid)
interior(ϕ) .= real.(ifft(ϕ̃, [1, 2]))

@show sum(interior(ϕ));

sum(interior(ϕ)) = -2.722266856380884e-13


In [44]:
#####
##### Compute Laplacian of solution ϕ to test that it's correct
#####

# Gotta fill halo regions
pbcs = HorizontallyPeriodicBCs()
fill_halo_regions!(ϕ.data, pbcs, arch, fake_grid)

∇²ϕ = CellField(Float64, arch, fake_grid)

for i in 1:Nx, j in 1:Ny, k in 1:Nz
    ∇²ϕ.data[i, j, k] = ∇²(i, j, k, Δx, Δy, ΔzF, ΔzC, ϕ.data)
end

In [45]:
interior(∇²ϕ) .- R

8×8×8 Array{Float64,3}:
[:, :, 1] =
  0.0747879    0.00768291   0.00693463  …  -0.0287096  -0.00206254
 -0.00729898  -0.0302827    0.0434758       0.0449527   0.00784128
  0.0128496   -0.0380171   -0.00668675     -0.0730191   0.028014  
 -0.0186299   -0.0243082    0.0592893       0.03326     0.0135834 
 -0.063507     0.0342042   -0.0497079       0.0118175   0.0496496 
  0.0209803   -0.0243121    0.00846879  …  -0.0380654  -0.0338457 
 -0.0394594   -0.0201254    0.0565744       0.0800537  -0.0102241 
 -0.0178961    0.0165818   -0.0164356      -0.0135414   0.0201495 

[:, :, 2] =
 -3.88578e-16  -1.38778e-17   1.38778e-17  …  -1.73472e-17   1.15359e-16
 -9.71445e-17  -5.55112e-17   4.16334e-17     -2.63678e-16  -2.77556e-17
 -5.55112e-17   5.55112e-17  -3.46945e-17      1.04083e-16   0.0        
  1.38778e-16   1.52656e-16   1.38778e-17      1.42247e-16   2.08167e-17
  6.93889e-18  -2.77556e-17   1.11022e-16      3.1225e-17   -9.71445e-17
  6.93889e-17   4.51028e-17   5.89806e-17  …   8.3